In [30]:
import requests
import pandas as pd
import numpy as np
import csv
import io
import datetime
from sqlalchemy import create_engine
import pyodbc
import urllib.request
import json
import pyodbc
from sqlalchemy import create_engine
import sqlalchemy
from pandas.io.json import json_normalize
import time
import dateparser
from googletrans import Translator
import timeit
import codecs
import pickle
import pythainlp

Clean data and make sure they are in the correct format

In [4]:
# open pickle of date
with open("unique_date_dict.pkl", 'rb') as f:
    temp = pickle.load(f)

In [5]:
#put it as DataFrame object
df_temp = pd.Series(temp).to_frame()
df_temp_2 = df_temp.reset_index()
df_temp_2

,index,0
0,-,-
1,1 ก.ค. 52,1 Jul 09
2,1 ก.ค. 57,1 Jul 14
3,1 ก.ค. 58,1 Jul 15
4,1 ก.ค. 59,1 Jul 16
...,...,...
3486,9 เม.ย. 62,9 Apr 19
3487,9 เม.ย. 63,9 Apr 63
3488,9 เม.ย. 64,9 Apr 64
3489,9 เม.ย. 65,9 Apr 65


In [6]:
#create temp value
df_temp_2["temp_eng"] =  df_temp_2[0]

In [33]:
#break date value into it own col
df_temp_2['day'], df_temp_2['month'], df_temp_2['year'] = df_temp_2['temp_eng'].str.split(' ', 2).str

C:\Users\v-krvibh\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


In [34]:
#replace &#39; with ''
df_temp_2['year'].replace('&#39;', '', regex=True,inplace = True)


In [44]:
#if 2 digits add 20 to the front

def add_20(x):
    if isinstance(x , str):
        if ( len(x) <= 2):
            x = '20' + x
    return x

In [45]:
df_temp_2['year'].unique()

array([nan, '09', '14', '15', '16', '17', '18', '19', '63', '64', '65',
       '66', '67', '89', '11', '2014', '61', '10', '2015', '62', '68',
       '88', '00', '44', '82', '95', '2018', '13', '12', '2013', '2017',
       '91', '2019', '08', '92', '79', '75', '23', '24', '85', '90', '30',
       '06', '86', '26', '74', '29', '56', '07', '99', '69', '20', '2003',
       '72', '31', '57', '71', '2016', '46', '70', '87', '96', '28', '01',
       '21', '73', '77', '37', '27', '32', '83', '78'], dtype=object)

In [46]:
df_temp_2['year'] = df_temp_2['year'].apply(add_20)

In [47]:
df_temp_2.year.unique()

array([nan, '2009', '2014', '2015', '2016', '2017', '2018', '2019',
       '2063', '2064', '2065', '2066', '2067', '2089', '2011', '2061',
       '2010', '2062', '2068', '2088', '2000', '2044', '2082', '2095',
       '2013', '2012', '2091', '2008', '2092', '2079', '2075', '2023',
       '2024', '2085', '2090', '2030', '2006', '2086', '2026', '2074',
       '2029', '2056', '2007', '2099', '2069', '2020', '2003', '2072',
       '2031', '2057', '2071', '2046', '2070', '2087', '2096', '2028',
       '2001', '2021', '2073', '2077', '2037', '2027', '2032', '2083',
       '2078'], dtype=object)

In [48]:
def change_year(x):
    if isinstance(x , str):
        if ( int(x) > 2050):
            x = int(x) - 43
    return str(x)

In [50]:
df_temp_2['year'] = df_temp_2['year'].apply(change_year)

In [55]:
df_temp_2

,index,0,temp_eng,day,month,year
0,-,-,-,-,NaN,nan
1,1 ก.ค. 52,1 Jul 09,1 Jul 09,1,Jul,2009
2,1 ก.ค. 57,1 Jul 14,1 Jul 14,1,Jul,2014
3,1 ก.ค. 58,1 Jul 15,1 Jul 15,1,Jul,2015
4,1 ก.ค. 59,1 Jul 16,1 Jul 16,1,Jul,2016
...,...,...,...,...,...,...
3486,9 เม.ย. 62,9 Apr 19,9 Apr 19,9,Apr,2019
3487,9 เม.ย. 63,9 Apr 63,9 Apr 63,9,Apr,2020
3488,9 เม.ย. 64,9 Apr 64,9 Apr 64,9,Apr,2021
3489,9 เม.ย. 65,9 Apr 65,9 Apr 65,9,Apr,2022


In [56]:
df_temp_2['Eng_date_cleaned'] = df_temp_2['day'] + ' ' + df_temp_2['month'] + ' ' + df_temp_2['year']

In [57]:
df_temp_2

,index,0,temp_eng,day,month,year,Eng_date_cleaned
0,-,-,-,-,NaN,nan,NaN
1,1 ก.ค. 52,1 Jul 09,1 Jul 09,1,Jul,2009,1 Jul 2009
2,1 ก.ค. 57,1 Jul 14,1 Jul 14,1,Jul,2014,1 Jul 2014
3,1 ก.ค. 58,1 Jul 15,1 Jul 15,1,Jul,2015,1 Jul 2015
4,1 ก.ค. 59,1 Jul 16,1 Jul 16,1,Jul,2016,1 Jul 2016
...,...,...,...,...,...,...,...
3486,9 เม.ย. 62,9 Apr 19,9 Apr 19,9,Apr,2019,9 Apr 2019
3487,9 เม.ย. 63,9 Apr 63,9 Apr 63,9,Apr,2020,9 Apr 2020
3488,9 เม.ย. 64,9 Apr 64,9 Apr 64,9,Apr,2021,9 Apr 2021
3489,9 เม.ย. 65,9 Apr 65,9 Apr 65,9,Apr,2022,9 Apr 2022


In [76]:
df_temp_3 = df_temp_2[['index','Eng_date_cleaned']]

In [78]:
unique_date_dict_cleaned = {}
unique_date_dict_key = df_temp_3['index']
unique_date_dict_value = df_temp_3['Eng_date_cleaned']

In [79]:
res = dict(zip(unique_date_dict_key, unique_date_dict_value))

In [81]:
f = open("unique_date_dict_cleaned.pkl","wb")
pickle.dump(res,f)
f.close()

In [3]:
df_contract = pd.read_csv("contract_eng_consolidated.csv")
df_project = pd.read_csv("project_eng_consolidated.csv")

C:\Users\v-krvibh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:
df_project.head()

,project_type_name_eng,purchase_method_group_name_eng,province_eng,project_status_eng,district_eng,announce_date_eng,transaction_date_eng,project_money_cleaned,price_build_cleaned,sum_price_agree_cleaned,dept_name_eng,budget_year_cleaned,project_id,project_name,geom
0,Hire for construction,Bidding,Nonthaburi,In progress,Bang Kruai,2 Apr 2013,9 March 2015,0,0,70624000000,Electricity Generating Authority of Thailand,2015,56045006726,ประกวดราคาจ้างซื้อและจ้างก่อสร้างโรงไฟฟ้าเพื่อ...,POINT(100.498178770987 13.8068549692447)
1,Outsourcing / Contracting services,Bidding,Nonthaburi,In progress,Bang Kruai,21 Nov 2014,26 Aug 2015,30000000000,26910000000,22871000000,Electricity Generating Authority of Thailand,2015,57115200071,ประกวดราคาจ้างเหมาขุด-ขนดินและถ่าน ที่เหมืองแม...,POINT(100.498178770987 13.8068549692447)
2,Outsourcing / Contracting services,E-bidding,Bangkok,In progress,Din,27 Aug 2014,13 Oct 2014,4455190000,4480010000,4436575000,Bangkok,2015,57085094977,ประกวดราคาจ้างเหมาเอกชนขนมูลฝอยจากศูนย์กำจัดมู...,POINT(100.565063509801 13.7780980259196)
3,buy,Special,Bangkok,In progress,Dusit,NaN,28 Jan 2015,2800000000,2799999972,2725935375,Army,2015,57085194402,ซื้อฮ.ท.ขนาดเบา แบบที่ 2 จำนวน 6 เครื่อง โดยวิ...,POINT(100.519873059321 13.7873113160748)
4,Hire for construction,E-bidding,Bangkok,In progress,Chatuchak,25 Mar 2014,9 Oct 2014,2599999769,2599999769,2588950000,Army,2015,57035180671,ประกวดราคาจ้างก่อสร้างอาคารเฉลิมพระเกียรติ สมเ...,POINT(100.584152190858 13.8453156541146)
